<a href="https://colab.research.google.com/github/mjRam27/Explorix-AI/blob/main/ExplorixAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir -p /content/drive/MyDrive/explorix


In [3]:
!mv /explorix_llama3b_finetune_200.jsonl /content/drive/MyDrive/explorix/


In [4]:
!ls /content/drive/MyDrive/explorix


explorix_llama3b_finetune_200.jsonl


In [5]:
!pip install transformers accelerate bitsandbytes peft trl datasets sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 25.2 MB/s eta 0:00:00


In [6]:
from huggingface_hub import login
login()


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "meta-llama/Llama-3.2-3B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [8]:
!nvidia-smi


Mon Dec 29 01:41:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             26W /   70W |    3228MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

MODEL_NAME = "meta-llama/Llama-3.2-3B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    use_fast=False,
    use_auth_token=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    use_auth_token=True
)

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False


/usr/local/lib/python3.12/dist-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 4,587,520 || all params: 3,217,337,344 || trainable%: 0.1426


In [13]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="/content/drive/MyDrive/explorix/explorix_llama3b_finetune_200.jsonl"
)

def format_prompt(example):
    messages = example["messages"]

    system_msg = ""
    user_msg = ""
    assistant_msg = ""

    for msg in messages:
        if msg["role"] == "system":
            system_msg = msg["content"]
        elif msg["role"] == "user":
            user_msg = msg["content"]
        elif msg["role"] == "assistant":
            assistant_msg = msg["content"]

    return f"""### System:
{system_msg}

### User:
{user_msg}

### Assistant:
{assistant_msg}"""


In [12]:
print(dataset["train"][0])


{'messages': [{'role': 'system', 'content': 'You are Explorix AI, a travel and exploration assistant built by Manoj Padmanabha.\n\nYou help users understand journeys, transportation options, and places to explore in a calm, enthusiastic, and adventurous tone.\n\nYour responses should feel friendly, inspiring, and reliable.\nDo not mention internal systems, databases, tables, or technical implementation details.\nIf information is missing or uncertain, clearly explain the limitation without guessing.\nDo not claim to be ChatGPT or any other assistant.'}, {'role': 'user', 'content': 'Who are you?'}, {'role': 'assistant', 'content': 'I’m Explorix AI, your travel and exploration companion, here to guide your journeys with clarity and confidence.'}]}


In [14]:
dataset = dataset["train"].map(
    lambda x: {"text": format_prompt(x)},
    remove_columns=dataset["train"].column_names
)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [15]:
print(dataset[0]["text"])


### System:
You are Explorix AI, a travel and exploration assistant built by Manoj Padmanabha.

You help users understand journeys, transportation options, and places to explore in a calm, enthusiastic, and adventurous tone.

Your responses should feel friendly, inspiring, and reliable.
Do not mention internal systems, databases, tables, or technical implementation details.
If information is missing or uncertain, clearly explain the limitation without guessing.
Do not claim to be ChatGPT or any other assistant.

### User:
Who are you?

### Assistant:
I’m Explorix AI, your travel and exploration companion, here to guide your journeys with clarity and confidence.


In [26]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


In [27]:
def tokenize(batch):
    tokenized = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

    # ✅ THIS IS REQUIRED FOR LOSS
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [30]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./explorix-qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


In [28]:
tokenized_dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=dataset.column_names
)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [29]:
tokenized_dataset[0].keys()


dict_keys(['input_ids', 'attention_mask', 'labels'])

In [31]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,5.343300
20,0.752600
30,0.418400
40,0.168800
50,0.099000
60,0.085200
70,0.080100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=75, training_loss=0.9307155605157217, metrics={'train_runtime': 529.6607, 'train_samples_per_second': 1.133, 'train_steps_per_second': 0.142, 'total_flos': 5203970909798400.0, 'train_loss': 0.9307155605157217, 'epoch': 3.0})

In [32]:
!cp -r ./explorix-qlora /content/drive/MyDrive/explorix/


In [35]:
!ls /content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75


adapter_config.json	   README.md	  scheduler.pt
adapter_model.safetensors  rng_state.pth  trainer_state.json
optimizer.pt		   scaler.pt	  training_args.bin


In [36]:
!zip -r explorix-qlora-checkpoint-75.zip \
/content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75


  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/ (stored 0%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/README.md (deflated 65%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/adapter_model.safetensors (deflated 8%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/adapter_config.json (deflated 57%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/training_args.bin (deflated 54%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/optimizer.pt (deflated 8%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/scheduler.pt (deflated 62%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/scaler.pt (deflated 64%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/rng_state.pth (deflated 26%)
  adding: content/drive/MyDrive/explorix/explorix-qlora/checkpoint-75/trainer_state.json (deflated 67%)


In [37]:
from google.colab import files
files.download("explorix-qlora-checkpoint-75.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>